## data

In [ ]:
import pandas as pd
df_hr = pd.read_csv('./csv/hospital_reviews.csv')
df_hr_text = df_hr[['text']].dropna()
df_khr = pd.read_csv('./csv/Seleniums_kakao_hospital_reviews_modified.csv')
df_khr.dropna(inplace=True)

In [ ]:
df_hr_text[0:500]

In [ ]:
df_khr_extract_sec = df_khr.query('positive != 1')[['text', 'positive']]

In [ ]:
df_khr_extract_sec

In [ ]:
pip install --upgrade pandas

In [ ]:
# data = {
#     'text': ['쓰레기', '쓰레기병원', '개별로임'],
#     'Value': [0,0,0]
# }
# df_khr_extract_sec.append(data, ignore_index=True)

## tokenizing

In [ ]:
!pip install python-mecab-ko

In [ ]:
from mecab import MeCab
mecab = MeCab()

In [ ]:
mecab.pos(' 👍 👍 👍')


In [ ]:
import re
def regex_func(text) :
    patterns = '\\\\n'
    text_regex = re.sub(pattern=patterns,repl=' ',string=text)
    return text_regex
df_khr_extract_sec['text'] = df_khr_extract_sec['text'].apply(regex_func)
df_khr_extract_sec['text']

In [ ]:
mecab.pos(df_khr_extract_sec['text'].loc[495])

In [ ]:
def extract(series) :
    morphs_nouns_list = []
    for sentence in series:
        morphs_nouns = ''
        for morph, tag in mecab.pos(sentence) : 
            if tag in ['NNG', 'NNP', 'NNB', 'NNBC', 'NP', 'NR', 'VA', 'MAG', 'SY'] :
                morphs_nouns += (morph+' ')            
        morphs_nouns_list.append(morphs_nouns)
        pass
    return morphs_nouns_list
morphs_nouns_list = extract(df_khr_extract_sec['text'])

In [ ]:
df_khr_extract_sec['text_tokenized'] = morphs_nouns_list

In [ ]:
df_khr_extract_sec

In [ ]:
df_khr_extract_sec

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfVectorizer = TfidfVectorizer(tokenizer=None, ngram_range=(1,2))

In [ ]:
tfidfVectorizer.fit(df_khr_extract_sec['text_tokenized'])

In [ ]:
features = df_khr_extract_sec['text_tokenized']
target = df_khr_extract_sec['positive']

In [ ]:
tokenized_features = tfidfVectorizer.transform(features)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_jobs=-1)

In [ ]:
from sklearn.model_selection import train_test_split
features_train, features_test, target_train, target_test = train_test_split(tokenized_features, target, test_size=0.1, random_state=55)
features_train.shape, features_test.shape, target_train.shape, target_test.shape

In [ ]:
model.fit(features_train,target_train)

In [ ]:
model.predict(features_test)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
target_predict = model.predict(features_test)
print(classification_report(target_test, target_predict))

In [ ]:
# make predict function
def sample_predict(features):
    tokenized_features = tfidfVectorizer.transform(extract(features))
    return model.predict(tokenized_features)
sample_predict(['아...진짜 진찰 진료 개떡같이 하네'])